# TensorFlow 使用简单的CNN识别手写数字

### 1. 下载mnist数据


mnist 是一个手写数字的图片数据库,每一张图片都是0到9中的单个数字。每一张都是抗锯齿(Anti-aliasing)的灰度图,图片大小28x28像素,数字部分被归一化为20*20大小,位于图片的中间位置,保持了原来形状的比例.

tensorflow 提供了一个input_data.py文件，专门用于下载mnist数据，通过下面的代码调用：

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
mnist = read_data_sets("MNIST_data/", one_hot=True)

https://s3.amazonaws.com/lasagne/recipes/datasets/mnist/
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


下载完后会在当前目录下看到 一个名为 'MNIST_data'的文件夹，里面是下载的mnist数据，四个压缩包。压缩包里的内容分别是：

|文件	                                     |内容|
|---------------------------|----|
|train-images-idx3-ubyte.gz	|训练集图片 - 55000 张 训练图片, 5000 张 验证图片|
|train-labels-idx1-ubyte.gz	|训练集图片对应的数字标签|
|t10k-images-idx3-ubyte.gz	|测试集图片 - 10000 张 图片|
|t10k-labels-idx1-ubyte.gz	|测试集图片对应的数字标签|


>因为网络问题无法访问 原来的 [SOURCE_URL](http://yann.lecun.com/exdb/mnist/), 所以根据[这里](https://stackoverflow.com/questions/33731875/tensorflow-ioerror-errno-socket-error-errno-104-connection-reset-by-peer)  的建议修改`anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py` 里的链接为'
https://s3.amazonaws.com/lasagne/recipes/datasets/mnist/'

In [3]:
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape,mnist.validation.labels.shape)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)


### 2.  实现CNN算法

网络结构：两个卷积层+一个全连接层。比较的简单的一个网络结构，但CNN的要点都有。

In [4]:
import tensorflow as tf

In [5]:
def weight_variables(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variables(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [6]:
# 定义卷积和池化
def conv2d(x, W):
    return tf.nn.conv2d(x,W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

定义网络结构

In [7]:
#准备参数
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,28,28,1])

In [8]:
# 第一卷积层
w_conv1 = weight_variables([5,5,1,32])
b_conv1 = bias_variables([32])
h_conv1 = conv2d(x_image, w_conv1) + b_conv1
h_conv1 = tf.nn.relu(h_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 第二卷积层
w_conv2 = weight_variables([5,5,32,64])
b_conv2 = bias_variables([64])
h_conv2 = conv2d(h_pool1, w_conv2) + b_conv2
h_conv2 = tf.nn.relu(h_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 第三层全连接层
w_fc1 = weight_variables([7*7*64, 1024])
b_fc1 = bias_variables([1024])
keep_prob = tf.placeholder(tf.float32)
input_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(input_flat, w_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 第五 softmax 层 输出
w_fc2 = weight_variables([1024, 10])
b_fc2  = bias_variables([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

定义 损失函数 和 优化器

In [12]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(1e-4).minimize(cross_entropy)

定义 准确率

In [13]:
correct_prediction= tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

开始训练

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
batch_size=50

for i in range(batch_size*400):
    batch = mnist.train.next_batch(batch_size)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step%d, training accuracy %g"%(i, train_accuracy))
        
    train_step.run({x:batch[0], y_: batch[1], keep_prob:0.5})

step0, training accuracy 0.12
step100, training accuracy 0.18
step200, training accuracy 0.16
step300, training accuracy 0.3
step400, training accuracy 0.34
step500, training accuracy 0.34
step600, training accuracy 0.44
step700, training accuracy 0.58
step800, training accuracy 0.48
step900, training accuracy 0.58
step1000, training accuracy 0.72
step1100, training accuracy 0.66
step1200, training accuracy 0.64
step1300, training accuracy 0.56
step1400, training accuracy 0.6
step1500, training accuracy 0.7
step1600, training accuracy 0.7
step1700, training accuracy 0.74
step1800, training accuracy 0.64
step1900, training accuracy 0.76
step2000, training accuracy 0.7
step2100, training accuracy 0.64
step2200, training accuracy 0.8
step2300, training accuracy 0.8
step2400, training accuracy 0.8
step2500, training accuracy 0.74
step2600, training accuracy 0.78
step2700, training accuracy 0.84
step2800, training accuracy 0.76
step2900, training accuracy 0.82
step3000, training accuracy 0.

测试

In [1]:
print(accuracy.eval({x:mnist.test.images, y_: mnist.test.labels, keep_prob:1}))

NameError: name 'accuracy' is not defined